Mount the drive that contains the data.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import and load all the required packages.

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import torch
import os


Resize all the images to 224x224 and add a preprocessing layer.

In [ ]:
IMAGE_SIZE = [224, 224]

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

Specify the path to the file containing the data.
Split the available data into training data, testing data and validation data and reset the index.

In [ ]:
path = "/content/drive/MyDrive/Mixed Images"
filenames = os.listdir(path)

df=pd.DataFrame({'filename':filenames})
df["category"] = df.apply(lambda x: x['filename'].split('_')[0], axis=1)

temp1 = df[df.category=='Damaged']
temp2 = df[df.category=='Undamaged']
df = pd.concat([temp1, temp2],ignore_index=True, axis = 0)
df.category.value_counts()

train_df, validate_df = train_test_split(df, test_size=0.30, random_state=42, stratify=df["category"])
validate_df, test_df = train_test_split(validate_df, test_size=0.5, random_state=42, stratify=validate_df["category"])

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
print(train_df.category.value_counts())
print(validate_df.category.value_counts())
print(test_df.category.value_counts())

Image augmentation using the Image data generator function.

In [ ]:
train_datagen = ImageDataGenerator( rotation_range=15,
                                    rescale=1./255,
                                    shear_range=0.1,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip = True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1)

train_set = train_datagen.flow_from_dataframe(train_df,path,x_col='filename',y_col='category',
                                              target_size=(224, 224),class_mode='categorical',batch_size=32)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_set = validation_datagen.flow_from_dataframe(validate_df,path,x_col='filename',
                                                        y_col='category',target_size=(224, 224),
                                                        class_mode='categorical',batch_size=32,
                                                        seed = 342)

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_dataframe(test_df,path,x_col='filename',
                                            y_col='category',target_size=(224, 224),
                                            class_mode='categorical',batch_size=32, 
                                            shuffle=False, seed = 342)

In [ ]:
# Add dropout layers

# x = Dense(1024, activation='relu')(vgg.output)
# x = Dense(512, activation='relu')(x)
x = Flatten()(vgg.output)

prediction = Dense(len(train_df.category.value_counts()), activation='sigmoid')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Fit the model and use early stopping callback to stop training once the model performance stops improving.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, restore_best_weights=True)

r = model.fit(train_set,
              validation_data=validation_set,
              epochs=18,
              steps_per_epoch=len(train_set),
              validation_steps=len(validation_set),
              callbacks=[callback])

Plot loss and accuracy graphs to visualize the performance of the model.

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/VGG_ValLoss_2.png')
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig('/content/drive/MyDrive/VGG_ValACC_2.png')
plt.show()

Save the trained model on the drive.

In [ ]:
model.save('/content/drive/MyDrive/VGG16_2.h5')

In [ ]:
train_set.class_indices.items()

Load the saved model and make predictions on the test dataset.

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/VGG16/VGG16_2.h5')
test_set.reset()
pred = model.predict(test_set)
test_df["pred"] = np.argmax(pred, axis=1)
test_df["pred"] = test_df["pred"].replace({0:'Damaged',1:'Undamaged'})
test_df.head(20)

In [ ]:
mis = []

In [ ]:
k = test_df['category'] != test_df['pred']
mis.append(test_df[k]['filename'])

In [ ]:
mis[0][:50]

In [ ]:
mis[0][50:]

In [ ]:
test_df.to_csv("/content/drive/MyDrive/VGG16/test_set.csv")

In [ ]:
model.evaluate(test_set, verbose=2)

Plot a confusion matrix to understand the model predictions.

In [ ]:
cm = confusion_matrix(test_df['category'], test_df['pred'])
sns.heatmap(cm, annot=True)

In [ ]:
target_names = ['Damaged', 'Undamaged']
classification_report(test_df['category'], test_df['pred'], target_names= target_names)


In [ ]:
#              precision    recall  f1-score   support

#     Damaged       0.79      0.72      0.75       234
#   Undamaged       0.73      0.80      0.76       226

#    accuracy                           0.76       460
#   macro avg       0.76      0.76      0.76       460
#weighted avg       0.76      0.76      0.76       460


Visualise the predictions made by the model.

In [ ]:
from keras.preprocessing import image

sample_test = test_df.sample(n=20).reset_index(drop=True)
plt.figure(figsize=(20, 20))
for index, row in sample_test.iterrows():
    filename = row['filename']
    pred = row['pred']
    img = image.load_img(path + "/" + filename, target_size=(224,224))
    plt.subplot(4, 5, index+1)
    plt.imshow(img)
    plt.xlabel(filename.split(' ')[0] + '(' + "{}".format(pred) + ')' )
plt.tight_layout()
plt.show()

Example of Image augmentation.

In [ ]:
# Image Augmentation
example_df = train_df.sample(n=1).reset_index(drop=True)
example_set = train_datagen.flow_from_dataframe(
    example_df, 
    path,
    x_col='filename',
    y_col='category',
    target_size=(224,224),
    class_mode='categorical'
)

plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_set:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
vgg = tf.keras.models.load_model('/content/drive/MyDrive/VGG16/VGG16_2.h5')

In [ ]:
vgg.summary()

In [ ]:
last_conv_layer = vgg.get_layer("block5_conv3")
last_conv_layer_model = tf.keras.Model(vgg.inputs, last_conv_layer.output)

In [ ]:
classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input
y = vgg.get_layer("block5_pool")(x)
y = Flatten()(y)
# y = Dense(4096, activation='relu')(y)
y = Dense(3, activation='sigmoid')(y)
classifier_model = tf.keras.Model(classifier_input, y)

Grad-CAM mapping

In [ ]:
def gradCam(image):  
  with tf.GradientTape() as tape:
      inputs = image[np.newaxis, ...]
      last_conv_layer_output = last_conv_layer_model(inputs)
      tape.watch(last_conv_layer_output)
      preds = classifier_model(last_conv_layer_output)
      top_pred_index = tf.argmax(preds[0])
      top_class_channel = preds[:, top_pred_index]

  grads = tape.gradient(top_class_channel, last_conv_layer_output)
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

  last_conv_layer_output = last_conv_layer_output.numpy()[0]
  pooled_grads = pooled_grads.numpy()
  for i in range(pooled_grads.shape[-1]):
      last_conv_layer_output[:, :, i] *= pooled_grads[i]

  gradcam = np.mean(last_conv_layer_output, axis=-1)
 
  gradcam = np.clip(gradcam, 0, np.max(gradcam)) / np.max(gradcam)
  gradcam = cv2.resize(gradcam, (224, 224))

  plt.imshow(image)
  plt.imshow(gradcam, alpha=0.5)

Guided Grad-CAM mapping

In [ ]:
def guidedGradCam(image):  
  with tf.GradientTape() as tape:
      last_conv_layer_output = last_conv_layer_model(image[np.newaxis, ...])
      tape.watch(last_conv_layer_output)
      preds = classifier_model(last_conv_layer_output)
      top_pred_index = tf.argmax(preds[0])
      top_class_channel = preds[:, top_pred_index]

  grads = tape.gradient(top_class_channel, last_conv_layer_output)[0]
  last_conv_layer_output = last_conv_layer_output[0]

  guided_grads = (
      tf.cast(last_conv_layer_output > 0, "float32")
      * tf.cast(grads > 0, "float32")
      * grads
  )

  pooled_guided_grads = tf.reduce_mean(guided_grads, axis=(0, 1))
  guided_gradcam = np.ones(last_conv_layer_output.shape[:2], dtype=np.float32)

  for i, w in enumerate(pooled_guided_grads):
      guided_gradcam += w * last_conv_layer_output[:, :, i]

  guided_gradcam = cv2.resize(guided_gradcam.numpy(), (224, 224))

  guided_gradcam = np.clip(guided_gradcam, 0, np.max(guided_gradcam))
  guided_gradcam = (guided_gradcam - guided_gradcam.min()) / (
      guided_gradcam.max() - guided_gradcam.min()
  )


  @tf.custom_gradient
  def guided_relu(x):
      def grad(dy):
          return tf.cast(dy > 0, "float32") * tf.cast(x > 0, "float32") * dy

      return tf.nn.relu(x), grad

  class GuidedBackprop:
      def __init__(self, model, layer_name: str):
          self.model = model
          self.layer_name = layer_name
          self.gb_model = self.build_guided_model()

      def build_guided_model(self):
          gb_model = tf.keras.Model(
              self.model.inputs, self.model.get_layer(self.layer_name).output
          )
          layers = [
              layer for layer in gb_model.layers[1:] if hasattr(layer, "activation")
          ]
          for layer in layers:
              if layer.activation == tf.keras.activations.relu:
                  layer.activation = guided_relu
          return gb_model

      def guided_backprop(self, image: np.ndarray):
          with tf.GradientTape() as tape:
              inputs = tf.cast(image, tf.float32)
              tape.watch(inputs)
              outputs = self.gb_model(inputs)
          grads = tape.gradient(outputs, inputs)[0]
          return grads

  gb = GuidedBackprop(vgg, "block5_conv3")

  saliency_map = gb.guided_backprop(image[np.newaxis, ...]).numpy()
  saliency_map = saliency_map * np.repeat(guided_gradcam[..., np.newaxis], 3, axis=2)

  saliency_map -= saliency_map.mean()
  saliency_map /= saliency_map.std() + tf.keras.backend.epsilon()
  saliency_map *= 0.25
  saliency_map += 0.5
  saliency_map = np.clip(saliency_map, 0, 1)
  saliency_map *= (2 ** 8) - 1
  saliency_map = saliency_map.astype(np.uint8)

  plt.imshow(saliency_map)

In [ ]:
path = "/content/drive/MyDrive/Misclassified"
filenames = os.listdir(path)

df=pd.DataFrame({'filename':filenames})

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import cv2

In [ ]:
plt.figure(figsize=(10, 10))
for index, row in df.iterrows():
    filename = row['filename']
    img = np.array(load_img(path + "/" + filename, target_size=(224,224)))
    if index > 19:
      break
    plt.subplot(4, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for index, row in df.iterrows():
    filename = row['filename']
    img = np.array(load_img(path + "/" + filename, target_size=(224,224)))
    if index > 19:
      break
    plt.subplot(4, 3, index+1)
    gradCam(img)
    plt.xlabel(filename)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for index, row in df.iterrows():
    filename = row['filename']  
    img = np.array(load_img(path + "/" + filename, target_size=(224,224)))
    if index > 19:
      break
    plt.subplot(5, 4, index+1)
    guidedGradCam(img)
    plt.xlabel(filename)
plt.tight_layout()
plt.show()